In [4]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from file_exchange import config
from file_exchange.network import Network
from file_exchange.peers_view import PeersView
from file_exchange.user import User
import gevent
from gevent.pool import Group
from datetime import datetime

[autoreload of routes.user failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.5/dist-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/usr/lib/python3.5/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.5/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 626, in _exec
  File "<frozen importlib._bootstrap_external>", line 661, in exec_module
  File "<frozen importlib._bootstrap_external>", line 767, in get_code
  File "<frozen importlib._bootstrap_external>", line 727, in source_to_code
  File "<frozen importlib._bootstrap>", line 222, in _call_with_frames_removed
  File "/home/adrien/Cloud/These/p2p-private-cloud/code/notebooks/routes/user.py", line 44


In [31]:
conf = config.default
conf['n_users'] = 10
conf

{'gossip_size': 20,
 'layer_threshold': 0.001,
 'minimum_node_availability': 0,
 'n_layers': 3,
 'n_rounds': 2,
 'n_users': 10,
 'period': datetime.timedelta(0, 1)}

In [32]:
address_book = PeersView(conf)
net = Network()
users = [None] * conf['n_users']
for i in range(conf['n_users']):
    users[i] = User(address_book, net, conf)

def output_view(view, conf):
    i = 0
    while True:
        print("Iteration #{} at time {}".format(i, datetime.now().time()))
        #print(view.get())
        gevent.sleep(conf['period'].total_seconds())
        i+=1
        
users_threads = Group()
users_threads.map(lambda u: u.loop(), users)
print("Before spawn")
output_thread = gevent.spawn(output_view, address_book, conf)

print("At join")
users_threads.join()
print("User thread joined")
gevent.kill(output_thread)


Hello!
Hello!
Hello!
Hello!
Hello!
Hello!
Hello!
Hello!
Hello!
Hello!
Returned
Returned
Returned
Returned
Returned
Returned
Returned
Returned
Returned
Returned
Before spawn
At join
User thread joined


In [38]:
import pandas as pd

df = pd.DataFrame([[1, 2], [2, 3]], columns=['a', 'b'])
df2 = df.sample(n=1)
df2.iloc[0] = [0, 0]
df

,a,b
0,1,2
1,2,3
